In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode,substring, length, udf
from pyspark.sql.types import *
from pyspark.sql import Row
from itertools import cycle
from pyspark.ml.regression import LinearRegression
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql import functions as F
from pyspark.sql import types as T

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
sqlDF = spark.sql("SELECT * FROM investments")
sqlDF.show(5)

+-----------------+----+---------------+
|raised_amount_usd|year|       category|
+-----------------+----+---------------+
|            1.5E7|2014|     E-Commerce|
|            1.4E7|2015|         Drones|
|            1.4E7|2015|  Manufacturing|
|        1870000.0|2011|             3D|
|        1870000.0|2011|Computer Vision|
+-----------------+----+---------------+
only showing top 5 rows



### Year Wise

In [4]:
SQLQUERY =  """
            SELECT CATEGORY, 
            CAST(YEAR AS INT), 
            SUM(RAISED_AMOUNT_USD) AS TOTAL, 
            CAST(SUM(RAISED_AMOUNT_USD) AS DECIMAL(30)) AS TOTAL_DEC 
            FROM INVESTMENTS GROUP 
            BY CATEGORY, YEAR 
            """
#  ORDER BY YEAR DESC, TOTAL DESC

In [5]:
sqlDF = spark.sql(SQLQUERY)
sqlDF.show(5)

+------------------+----+--------------+----------+
|          CATEGORY|YEAR|         TOTAL| TOTAL_DEC|
+------------------+----+--------------+----------+
|  Mobile Analytics|2010|       4.205E7|  42050000|
|Big Data Analytics|2013|  2.35683979E9|2356839790|
| Online Scheduling|2012|      9.5505E7|  95505000|
|          Creative|2012|  1.65391911E8| 165391911|
|              Apps|2008|6.3178257004E8| 631782570|
+------------------+----+--------------+----------+
only showing top 5 rows



In [6]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['YEAR'], outputCol = 'FEATURES')
featureDF = vectorAssembler.transform(sqlDF).select('CATEGORY', 'FEATURES', 'TOTAL')

featureDF.show(5)

+------------------+--------+--------------+
|          CATEGORY|FEATURES|         TOTAL|
+------------------+--------+--------------+
|  Mobile Analytics|[2010.0]|       4.205E7|
|Big Data Analytics|[2013.0]|  2.35683979E9|
| Online Scheduling|[2012.0]|      9.5505E7|
|          Creative|[2012.0]|  1.65391911E8|
|              Apps|[2008.0]|6.3178257004E8|
+------------------+--------+--------------+
only showing top 5 rows



In [7]:
f = featureDF.select('CATEGORY').distinct()


In [8]:
Categories = [row.CATEGORY for row in f.collect()]


In [9]:
len(Categories)

837

In [10]:
# Features matrix to predict the amount for the Year 2020

l =  [(2020,)]

rdd = sc.parallelize(l)
test = rdd.map(lambda x: Row(YEAR=x[0] ))
testDF = sqlContext.createDataFrame(test)

vectorAssembler = VectorAssembler(inputCols = ['YEAR'], outputCol = 'FEATURES')
vectorDF = vectorAssembler.transform(testDF).select('FEATURES')

In [11]:
#count = 0 
summaryInfo = []
columns = ['Category', 'Gradient', 'Intercept','RMSE', 'R2', 'Pediction']

statInfo = []
for category in Categories:
    #print(category)
    categoryDF=featureDF.filter(featureDF.CATEGORY == category)
    if(categoryDF.count() > 10):
        #count +=1
        lr = LinearRegression(featuresCol = 'FEATURES', labelCol='TOTAL', maxIter=10, regParam=0.3, elasticNetParam=0.8)
        lr_model = lr.fit(categoryDF)
        if (lr_model.summary.r2 >= .5):
            row=(category
                 ,lr_model.coefficients
                 ,lr_model.intercept
                 ,lr_model.summary.rootMeanSquaredError
                 ,lr_model.summary.r2
                 ,lr_model.transform(vectorDF).take(1)[0].prediction )
            summaryInfo.append(row)                        
summaryDF = spark.createDataFrame(summaryInfo, columns)


In [12]:
summaryDF.show()

+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|            Category|            Gradient|           Intercept|                RMSE|                R2|           Pediction|
+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|    Personal Finance|[3.09531844548871...|-6.20846579539486...|1.1403333171358914E9|0.5257188832329653|  4.40774644923291E9|
|Reviews and Recom...|[9.38435051053559...|-1.88076601008072...| 2.252600279853369E8|0.6748252913534014|1.4872793047460022E9|
|         Health Care|[1.08986421816095...|-2.17925088640816...|2.0237455217200394E9|0.8747611056595744|2.227483427696704E10|
|Application Perfo...|[1.46386390370929...|-2.93690139320376...|4.7040592338191056E8|0.5668998218851247|2.0103692289013062E9|
|        Credit Cards|[6.33419228608998...|-1.27009681219964...|2.5974312104301742E8|0.5027780036421106| 9.41002959053

In [13]:
summaryDF.write.mode("overwrite").saveAsTable("summaryDF")

In [15]:
SQLQUERY =  """
            SELECT *
            FROM summaryDF
            
            """
#  ORDER BY YEAR DESC, TOTAL DESC

sqlDF = spark.sql(SQLQUERY)
sqlDF.show(5)

+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|            Category|            Gradient|           Intercept|                RMSE|                R2|           Pediction|
+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|    Personal Finance|[3.09531844548871...|-6.20846579539486...|1.1403333171358914E9|0.5257188832329653|  4.40774644923291E9|
|Reviews and Recom...|[9.38435051053559...|-1.88076601008072...| 2.252600279853369E8|0.6748252913534014|1.4872793047460022E9|
|         Health Care|[1.08986421816095...|-2.17925088640816...|2.0237455217200394E9|0.8747611056595744|2.227483427696704E10|
|Application Perfo...|[1.46386390370929...|-2.93690139320376...|4.7040592338191056E8|0.5668998218851247|2.0103692289013062E9|
|        Credit Cards|[6.33419228608998...|-1.27009681219964...|2.5974312104301742E8|0.5027780036421106| 9.41002959053